<a href="https://colab.research.google.com/github/syafiqahmarsya/mine/blob/main/Copy_of_Capstone_Final_Project_AMS_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Capstone Final Project Analisis Media Sosial***

| NIM | Nama | Deskripsi |
| -------- | -------- | -------- |
|11220940000053  |USWATUN HASANAH| |
|11220940000056  |SYAFIQAH MARSYA KHOLIYADI ||
|11220940000063   |AWALIA DAMAYANTI  | |







## Latar Belakang

  Aksi boikot produk oleh netizen Indonesia di media sosial semakin masif dilakukan, dipicu oleh isu genosida yang dilakukan Israel terhadap palestina. Kampanye boikot tersebut kerap viral di berbagai platform media sosial salah satunya di YouTube, hal ini berdampak langsung pada reputasi brand dan kepercayaan konsumen yang dapat mempengaruhi  hasil penjualan. Perusahaan yang terdampak, seperti Starbucks, Unilever, MCD, dll., perlu memahami bagaimana sentimen publik berkembang agar dapat merespons secara tepat. Dalam hal ini, analisis sentimen netizen menjadi penting untuk menggambarkan persepsi publik, mengidentifikasi isu utama, serta memberikan rekomendasi dari insight yang diperoleh agar dapat digunakan oleh perusahaan dalam pengambilan keputusan untuk mitigasi dampak dari aksi boikot.

## Rumusan Masalah

**Permasalahan**

1. Bagaimana tren sentimen netizen terhadap produk yang terkena boikot di media sosial dalam periode tertentu?

2. Apa isu utama yang muncul dalam percakapan netizen terkait boikot tersebut?


3. Bagaimana distribusi sentimen (positif, netral, negatif) berdasarkan jenis produk, merek, atau tagar yang digunakan?


4. Apa saja insight dan rekomendasi bisnis yang dapat diberikan kepada perusahaan terkait agar dapat merespons atau memitigasi dampak dari boikot tersebut?



**Periode Pengumpulan Data**

17 Oktober 2023 - 15 Juli 2025

**Sumber Data**

YouTube

**Kata Kunci**
* #produkisrael
* #boikotprodukisrael
* #aksiboikot
* #penggantiprodukisrel
* #boikotkfc
* #boikotunilever
* #boikotstarbuck
* #dampakboikot



## Packages

In [1]:
!pip install sastrawi
!pip install emoji # Install the emoji library
!pip install textblob
from textblob import TextBlob
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import emoji

nltk.download('popular')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

# **Mempersiapkan Data**

## Pengambilan Data YouTube

Kami menggunakan 8 keyword dan melakukan proses pengambilan komentar di video youtobe secara acak menggunakan API




```
# Pengambilan yang kami lakukan
import csv
from datetime import datetime
import googleapiclient.discovery

api_key = "your-api-key"

keywords = ['#produkisrael',
'#boikotprodukisrael',
'#aksiboikot',
'#penggantiprodukisrael',
"#boikotkfc",
'#boikotunilever',
'#boikotstarbuck',
'#dampakboikot']

used_video_ids = set()  # Untuk menyimpan video yang sudah dipakai

def search_and_get_comments(search_query, max_videos=10, max_comments_per_video=250, start_date=None, end_date=None, csv_filename="youtube_comments.csv"):
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

    search_response = youtube.search().list(
        part="snippet",
        q=search_query,
        type="video",
        maxResults=30  # ambil lebih banyak untuk jaga-jaga (nanti kita filter jadi 10 unik)
    ).execute()

    # Filter video yang belum dipakai
    video_titles_and_ids = {}
    for item in search_response['items']:
        if item['id']['kind'] == 'youtube#video':
            video_id = item['id']['videoId']
            if video_id not in used_video_ids:
                video_titles_and_ids[video_id] = item['snippet']['title']
                if len(video_titles_and_ids) >= max_videos:
                    break

    used_video_ids.update(video_titles_and_ids.keys())  # Tandai video ini sudah terpakai

    if not video_titles_and_ids:
        print(f"No new videos found for keyword '{search_query}'. Skipping.")
        return

    total_comments = 0
    unique_comments = set()

    start_dt = datetime.strptime(start_date, "%Y-%m-%d") if start_date else None
    end_dt = datetime.strptime(end_date, "%Y-%m-%d") if end_date else None

    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["keyword", "video_id", "video_title", "comment_text", "published_at"])  # Header

        for video_id, video_title in video_titles_and_ids.items():
            try:
                next_page_token = None
                comments_written = 0

                while True:
                    request = youtube.commentThreads().list(
                        part="snippet",
                        videoId=video_id,
                        maxResults=100,
                        pageToken=next_page_token
                    )
                    response = request.execute()

                    for item in response['items']:
                        try:
                            snippet = item['snippet']['topLevelComment']['snippet']
                            comment_text = snippet['textDisplay']
                            published_at = snippet['publishedAt']
                            comment_dt = datetime.strptime(published_at, "%Y-%m-%dT%H:%M:%SZ")

                            if (start_dt and comment_dt < start_dt) or (end_dt and comment_dt > end_dt):
                                continue

                            unique_id = f"{video_id}_{comment_text}"
                            if unique_id in unique_comments:
                                continue
                            unique_comments.add(unique_id)

                            writer.writerow([search_query, video_id, video_title, comment_text, published_at])
                            total_comments += 1
                            comments_written += 1

                            if comments_written >= max_comments_per_video:
                                break
                        except KeyError:
                            continue

                    next_page_token = response.get("nextPageToken")
                    if not next_page_token or comments_written >= max_comments_per_video:
                        break

            except Exception as e:
                print(f"Error retrieving comments for video ID {video_id}: {e}")

    print(f"CSV file '{csv_filename}' created for keyword '{search_query}'.")
    print(f"Total unique comments retrieved: {total_comments}")
    print(f"Number of videos fetched: {len(video_titles_and_ids)}")

# Run for all keywords
start_date = "2023-10-7"
end_date = "2025-07-15"

for keyword in keywords:
    csv_file = f"youtube_comments_{keyword.replace(' ', '_')}.csv"
    search_and_get_comments(keyword, max_videos=10, max_comments_per_video=250, start_date=start_date, end_date=end_date, csv_filename=csv_file)

     
```




Setelah mengambil data di YouTube lalu kami lakukan proses penggabungan dan buang komentar duplikat agar satu keyword mengambil video yang berbeda dari yang lain.


```
import pandas as pd
import glob

# Mencari semua file CSV di dalam folder 'tweets-data'
csv_files = glob.glob('tweets-data/*.csv') # list nama file yang disimpan
csv_files.sort()  # Use the sort() method of the list object

n = 0
for filename in csv_files: ## Menggabungkan file csv dalam bentuk dataframe
  # Read the CSV file into a pandas DataFrame
  if n == 0:
    df = pd.read_csv(filename, delimiter=",")
    n = 1
  else:
    df2 = pd.read_csv(filename, delimiter=",")
    df = pd.concat([df, df2])
    n+=1

# Membuang duplikat berdasarkan id_str
df.drop_duplicates(subset=['id_str'], keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)

# Ekspor Data Hasil Scrapping
from google.colab import files
namafile = "DATAX_Naturalisasi_fix.csv"
df.to_csv(namafile, index=False)
files.download(namafile)
"Exported"
```



## Import Data

Berikut Link hasil pengambilan data dari youtobe:
- https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/all_youtube_comments.csv


In [2]:
# Import data hasil pengambilan dari youtobe
import pandas as pd

# URL file raw CSV dari GitHub
url = 'https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/all_youtube_comments.csv'

# Membaca file langsung dari URL
dataRaw = pd.read_csv(url, low_memory=False, encoding='utf8')

# Menampilkan beberapa baris pertama
dataRaw.head()

,keyword,video_id,video_title,comment_text,published_at
0,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Starbucks dukung LGBT.mari boikot,2025-07-09T07:39:42Z
1,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"starbug itu kopi shop apa an sich,gk jelas.......",2025-06-21T08:28:00Z
2,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"imigran timteng memang gila, berhasil menarik ...",2025-06-20T00:33:48Z
3,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Kmu beli satu kopi sama aja mengirim 1 peluru ...,2025-06-10T17:17:52Z
4,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,The power of key is stand with palestine or le...,2025-04-10T10:00:58Z


In [3]:
# melihat kolom di dataRaw
dataRaw.columns

Index(['keyword', 'video_id', 'video_title', 'comment_text', 'published_at'], dtype='object')

## Preprocessing

### Menambahkan labelling ke dalam Dataset

Dari data komentar yang telah didapatkan, kami labelling dan buang komentar yang tidak berbahasa indonesia serta komentar terindikasi judi online. Selanjutnya data tersebut disimpan di link github berikut :

https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/youtube_comments_labelling_no_judol.csv

Kami telah melakukan proses pelabelan sentimen (1:Positif / mendukudung penggunaan produk terboikot, 2:Netral, dan 3:Negatif / tidak mendukung penggunaan produk terboikot) pada setiap komentar sebelum data ini diunggah ke GitHub dan diimpor ke sini. Langkah ini dilakukan untuk mendukung analisis sentimen publik terkait topik Penggunaan Produk yang Terkena Boikot di Media Sosial

In [4]:
url2 = "https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/youtube_comments_labelling_no_judol.csv"
data = pd.read_csv(url2, low_memory=False, encoding='utf8')
data

,keyword,video_id,video_title,comment_text,published_at,label,Unnamed: 6
0,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Starbucks dukung LGBT.mari boikot,2025-07-09T07:39:42Z,3,NaN
1,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"starbug itu kopi shop apa an sich,gk jelas.......",2025-06-21T08:28:00Z,3,NaN
2,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"imigran timteng memang gila, berhasil menarik ...",2025-06-20T00:33:48Z,2,NaN
3,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Kmu beli satu kopi sama aja mengirim 1 peluru ...,2025-06-10T17:17:52Z,3,NaN
4,#boikotstarbuck,07TnYABFVXI,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Saya nggak tau bagaimana kok Starbucks bisa gi...,2025-03-21T01:15:54Z,3,NaN
...,...,...,...,...,...,...,...
6147,#boikotkfc,4pyU3-Ry5ks,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Pertanyaanya Indonesia kapan,2024-04-30T17:29:06Z,2,NaN
6148,#boikotkfc,4pyU3-Ry5ks,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Indo apa kabar ? Masih suka kah melihat anak2 ...,2024-04-30T16:25:40Z,2,NaN
6149,#boikotkfc,4pyU3-Ry5ks,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,❤🇮🇱🇮🇱❤ ane boikot makanan dan restoran palestina,2024-04-30T16:04:55Z,3,NaN
6150,#boikotkfc,4pyU3-Ry5ks,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Orang +62 nggak malu sama Malaysia yang udah b...,2024-04-30T16:01:26Z,3,NaN


In [5]:
# Drop kolom yang tidak diperlukan
data = data.drop(columns=['video_id'])
# Ubah urutan kolom: published_at di awal, sisanya diikuti
cols = ['published_at', 'keyword', 'video_title', 'comment_text', 'label']
data = data[cols]

data

,published_at,keyword,video_title,comment_text,label
0,2025-07-09T07:39:42Z,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Starbucks dukung LGBT.mari boikot,3
1,2025-06-21T08:28:00Z,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"starbug itu kopi shop apa an sich,gk jelas.......",3
2,2025-06-20T00:33:48Z,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"imigran timteng memang gila, berhasil menarik ...",2
3,2025-06-10T17:17:52Z,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Kmu beli satu kopi sama aja mengirim 1 peluru ...,3
4,2025-03-21T01:15:54Z,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Saya nggak tau bagaimana kok Starbucks bisa gi...,3
...,...,...,...,...,...
6147,2024-04-30T17:29:06Z,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Pertanyaanya Indonesia kapan,2
6148,2024-04-30T16:25:40Z,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Indo apa kabar ? Masih suka kah melihat anak2 ...,2
6149,2024-04-30T16:04:55Z,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,❤🇮🇱🇮🇱❤ ane boikot makanan dan restoran palestina,3
6150,2024-04-30T16:01:26Z,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Orang +62 nggak malu sama Malaysia yang udah b...,3


### Duplikasi Data

In [6]:
# Menampilkan comment text yang duplikat
duplicate_count = data.duplicated(subset='comment_text').sum()
print(f"Jumlah komentar duplikat: {duplicate_count}")

Jumlah komentar duplikat: 49


In [7]:
# Menampilkan contoh komentar duplikat dalam bentuk tabel
data[data.duplicated(subset='comment_text', keep=False)].head(86)

,published_at,keyword,video_title,comment_text,label
105,2024-06-14T06:49:45Z,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,NaN,2
114,2024-06-13T14:54:17Z,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,😂😂😂😂😂😂,2
117,2024-06-12T23:30:45Z,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Tetap boikot,3
268,2024-06-06T07:45:31Z,#boikotstarbuck,"Massa Gerebek Starbucks, Serukan Bela Palestin...",Lanjutkan,3
290,2024-05-13T04:56:47Z,#boikotstarbuck,Saham Starbucks Anjlok Akibat Boikot! #boikot ...,👍👍👍,2
...,...,...,...,...,...
5961,2024-05-04T05:00:54Z,#boikotkfc,KFC TUTUP BUKAN DISEBABKAN BOIKOT,Alhamdulillah,2
5962,2024-05-04T04:52:18Z,#boikotkfc,KFC TUTUP BUKAN DISEBABKAN BOIKOT,😂😂😂😂,2
6037,2025-06-10T03:40:36Z,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Bagus,2
6076,2024-05-08T17:45:30Z,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Alhamdulillah,2


In [8]:
# Menghapus Duplikat
data.drop_duplicates(subset='comment_text', inplace=True)

In [9]:
# Cek kembali yang Duplikat
duplicate_count = data.duplicated(subset='comment_text').sum()
print(f"Jumlah komentar duplikat: {duplicate_count}")

Jumlah komentar duplikat: 0


In [10]:
# Menyimpan dalam bentuk CSV untuk data tanpa komen yanng duplikat
# dataRaw.to_csv('comments_no_duplicates.csv', index=False)

### Ubah Tipe Data

In [11]:
# Cek tipe data
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6103 entries, 0 to 6151
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   published_at  6103 non-null   object
 1   keyword       6103 non-null   object
 2   video_title   6103 non-null   object
 3   comment_text  6102 non-null   object
 4   label         6103 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 286.1+ KB


In [12]:
# Ubah 'keyword' dan 'label' menjadi kategori
data['label'] = data['label'].astype('category')
data['keyword'] = data['keyword'].astype('category')

# Ubah 'published_at' menjadi datetime (format ISO 8601 ke waktu biasa)
data['published_at'] = pd.to_datetime(data['published_at'], format='%Y-%m-%dT%H:%M:%SZ')

# Cek hasil
data.dtypes

,0
published_at,datetime64[ns]
keyword,category
video_title,object
comment_text,object
label,category


In [13]:
# Cek jumlah tiap label
data['label'].value_counts()

,count
label,
2,2654
3,1997
1,1452


### Missing Value

In [14]:
# Cek Missing Value
data.isnull().sum()

,0
published_at,0
keyword,0
video_title,0
comment_text,1
label,0


In [15]:
# Menghapus baris yang memiliki missing value
data = data.dropna()

# Mengecek ulang apakah masih ada missing value
print(data.isnull().sum())

published_at    0
keyword         0
video_title     0
comment_text    0
label           0
dtype: int64


# **Cleaning Text**

In [16]:
# Membersihkan teks dari URL, mention, emoji, hashtag, angka, tanda baca, dan spasi berlebih
import re
import string
import unicodedata
import emoji

def clean_text(text):
    text = text.lower()

    # Hilangkan karakter aneh (emoji, simbol asing)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Hapus URL, mention, hashtag
    text = re.sub(r'http\S+|www.\S+', '', text)       # URL
    text = re.sub(r'@\w+', '', text)                  # mention
    text = re.sub(r'#\w+', '', text)                  # hashtag

    # Hapus emoji
    text = emoji.replace_emoji(text, replace='')

    # Hapus angka
    text = re.sub(r'\d+', ' ', text)

    # Ganti semua tanda baca dengan spasi, bukan hapus
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)

    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [17]:
# Membersihkan semua teks dalam kolom comment_text
data['cleaned_text'] = data['comment_text'].apply(clean_text)
data

/tmp/ipython-input-17-257375155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_text'] = data['comment_text'].apply(clean_text)


,published_at,keyword,video_title,comment_text,label,cleaned_text
0,2025-07-09 07:39:42,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Starbucks dukung LGBT.mari boikot,3,starbucks dukung lgbt mari boikot
1,2025-06-21 08:28:00,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"starbug itu kopi shop apa an sich,gk jelas.......",3,starbug itu kopi shop apa an sich gk jelas pin...
2,2025-06-20 00:33:48,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"imigran timteng memang gila, berhasil menarik ...",2,imigran timteng memang gila berhasil menarik s...
3,2025-06-10 17:17:52,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Kmu beli satu kopi sama aja mengirim 1 peluru ...,3,kmu beli satu kopi sama aja mengirim peluru ke...
4,2025-03-21 01:15:54,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Saya nggak tau bagaimana kok Starbucks bisa gi...,3,saya nggak tau bagaimana kok starbucks bisa gi...
...,...,...,...,...,...,...
6147,2024-04-30 17:29:06,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Pertanyaanya Indonesia kapan,2,pertanyaanya indonesia kapan
6148,2024-04-30 16:25:40,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Indo apa kabar ? Masih suka kah melihat anak2 ...,2,indo apa kabar masih suka kah melihat anak tek...
6149,2024-04-30 16:04:55,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,❤🇮🇱🇮🇱❤ ane boikot makanan dan restoran palestina,3,ane boikot makanan dan restoran palestina
6150,2024-04-30 16:01:26,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Orang +62 nggak malu sama Malaysia yang udah b...,3,orang nggak malu sama malaysia yang udah beran...


## Normalisasi Data

In [18]:
# Buat folder data
!mkdir -p data
!wget -P data/ https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/Data_Slang_%26_Typo%20List.txt

# Lokasi file
slang_file = 'data/Data_Slang_&_Typo List.txt'

# Membuat kamus slang dari file
slang_dict = {}
with open(slang_file, 'r', encoding='utf-8') as file:
    for line in file:
        if ':' in line:
            slang, standard = line.strip().split(':', 1)
            slang_dict[slang.strip()] = standard.strip()

# Pastikan sudah import word_tokenize (dari NLTK)
from nltk.tokenize import word_tokenize

# Fungsi untuk mengganti slang dengan kata baku
def normalize_slang(text):
    words = word_tokenize(text.lower())
    return ' '.join([slang_dict.get(word, word) for word in words])

# Terapkan ke kolom 'cleaned_text'
data['normalized_comment'] = data['cleaned_text'].apply(normalize_slang)

# Lihat hasil
data[['cleaned_text', 'normalized_comment']].head()

--2025-07-17 17:21:28--  https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/Data_Slang_%26_Typo%20List.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82941 (81K) [text/plain]
Saving to: ‘data/Data_Slang_&_Typo List.txt.1’

Data_Slang_&_Typo L 100%[===================>]  81.00K  --.-KB/s    in 0.03s   

2025-07-17 17:21:28 (2.45 MB/s) - ‘data/Data_Slang_&_Typo List.txt.1’ saved [82941/82941]



/tmp/ipython-input-18-3672406363.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['normalized_comment'] = data['cleaned_text'].apply(normalize_slang)


,cleaned_text,normalized_comment
0,starbucks dukung lgbt mari boikot,starbucks dukung lgbt mari boikot
1,starbug itu kopi shop apa an sich gk jelas pin...,starbucks itu kopi shop apa an sih tidak jelas...
2,imigran timteng memang gila berhasil menarik s...,imigran timurtengah memang gila berhasil menar...
3,kmu beli satu kopi sama aja mengirim peluru ke...,kamu beli satu kopi sama saja mengirim peluru ...
4,saya nggak tau bagaimana kok starbucks bisa gi...,saya tidak tahu bagaimana sih starbucks bisa b...


In [19]:
# Cek isi data
data

,published_at,keyword,video_title,comment_text,label,cleaned_text,normalized_comment
0,2025-07-09 07:39:42,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Starbucks dukung LGBT.mari boikot,3,starbucks dukung lgbt mari boikot,starbucks dukung lgbt mari boikot
1,2025-06-21 08:28:00,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"starbug itu kopi shop apa an sich,gk jelas.......",3,starbug itu kopi shop apa an sich gk jelas pin...,starbucks itu kopi shop apa an sih tidak jelas...
2,2025-06-20 00:33:48,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"imigran timteng memang gila, berhasil menarik ...",2,imigran timteng memang gila berhasil menarik s...,imigran timurtengah memang gila berhasil menar...
3,2025-06-10 17:17:52,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Kmu beli satu kopi sama aja mengirim 1 peluru ...,3,kmu beli satu kopi sama aja mengirim peluru ke...,kamu beli satu kopi sama saja mengirim peluru ...
4,2025-03-21 01:15:54,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Saya nggak tau bagaimana kok Starbucks bisa gi...,3,saya nggak tau bagaimana kok starbucks bisa gi...,saya tidak tahu bagaimana sih starbucks bisa b...
...,...,...,...,...,...,...,...
6147,2024-04-30 17:29:06,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Pertanyaanya Indonesia kapan,2,pertanyaanya indonesia kapan,pertanyaanya indonesia kapan
6148,2024-04-30 16:25:40,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Indo apa kabar ? Masih suka kah melihat anak2 ...,2,indo apa kabar masih suka kah melihat anak tek...,indonesia apa kabar masih suka kah melihat ana...
6149,2024-04-30 16:04:55,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,❤🇮🇱🇮🇱❤ ane boikot makanan dan restoran palestina,3,ane boikot makanan dan restoran palestina,saya boikot makanan dan restoran palestina
6150,2024-04-30 16:01:26,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Orang +62 nggak malu sama Malaysia yang udah b...,3,orang nggak malu sama malaysia yang udah beran...,orang tidak malu sama malaysia yang sudah bera...


## Cleaning Data Lanjutan

In [20]:
# Membersihkan semua teks dalam kolom normalized_comment
data['cleaned_text2'] = data['normalized_comment'].apply(clean_text)
data

/tmp/ipython-input-20-3887304069.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_text2'] = data['normalized_comment'].apply(clean_text)


,published_at,keyword,video_title,comment_text,label,cleaned_text,normalized_comment,cleaned_text2
0,2025-07-09 07:39:42,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Starbucks dukung LGBT.mari boikot,3,starbucks dukung lgbt mari boikot,starbucks dukung lgbt mari boikot,starbucks dukung lgbt mari boikot
1,2025-06-21 08:28:00,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"starbug itu kopi shop apa an sich,gk jelas.......",3,starbug itu kopi shop apa an sich gk jelas pin...,starbucks itu kopi shop apa an sih tidak jelas...,starbucks itu kopi shop apa an sih tidak jelas...
2,2025-06-20 00:33:48,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"imigran timteng memang gila, berhasil menarik ...",2,imigran timteng memang gila berhasil menarik s...,imigran timurtengah memang gila berhasil menar...,imigran timurtengah memang gila berhasil menar...
3,2025-06-10 17:17:52,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Kmu beli satu kopi sama aja mengirim 1 peluru ...,3,kmu beli satu kopi sama aja mengirim peluru ke...,kamu beli satu kopi sama saja mengirim peluru ...,kamu beli satu kopi sama saja mengirim peluru ...
4,2025-03-21 01:15:54,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Saya nggak tau bagaimana kok Starbucks bisa gi...,3,saya nggak tau bagaimana kok starbucks bisa gi...,saya tidak tahu bagaimana sih starbucks bisa b...,saya tidak tahu bagaimana sih starbucks bisa b...
...,...,...,...,...,...,...,...,...
6147,2024-04-30 17:29:06,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Pertanyaanya Indonesia kapan,2,pertanyaanya indonesia kapan,pertanyaanya indonesia kapan,pertanyaanya indonesia kapan
6148,2024-04-30 16:25:40,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Indo apa kabar ? Masih suka kah melihat anak2 ...,2,indo apa kabar masih suka kah melihat anak tek...,indonesia apa kabar masih suka kah melihat ana...,indonesia apa kabar masih suka kah melihat ana...
6149,2024-04-30 16:04:55,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,❤🇮🇱🇮🇱❤ ane boikot makanan dan restoran palestina,3,ane boikot makanan dan restoran palestina,saya boikot makanan dan restoran palestina,saya boikot makanan dan restoran palestina
6150,2024-04-30 16:01:26,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Orang +62 nggak malu sama Malaysia yang udah b...,3,orang nggak malu sama malaysia yang udah beran...,orang tidak malu sama malaysia yang sudah bera...,orang tidak malu sama malaysia yang sudah bera...


### Stopword

Sebelumnya kita sudah gabungkan semua stopword menjadi satu file dalam link berikut :

https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/stopwords%20list.txt


In [21]:
!mkdir -p data
!wget -p data/stopwords_id.txt https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/stopwords%20list.txt
# Menghapus stopwords bahasa Indonesia
# Baca file stopwords
with open('data/stopwords_id.txt', "r", encoding="utf-8", errors='replace') as file:
    id_stop = file.readlines()

# Bersihkan stopwords (strip newline/spasi & lowercase)
id_stop = [word.strip().lower() for word in id_stop]

# Tambahkan kata-kata tambahan
additional_stopwords = {
    'nya', 'iya', 'itu', 'kah', 'dan', 'ke', 'di',
    'turun', 'naik', 'ambil', 'lihat',
    'tolong', 'atas', 'pokok', 'hari',
    'naikin', 'presiden', 'indonesia', 'sih', 'pakai',
    'biar', 'juta', 'ribu','tek'
}

# Gabungkan ke list utama
id_stop = set(id_stop).union(additional_stopwords)

# Fungsi hapus stopwords
def remove_custom_stopwords(text):
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in id_stop])

# Terapkan ke kolom yang sudah dibersihkan sebelumnya
data['no_stopwords'] = data['cleaned_text2'].apply(remove_custom_stopwords)

# Lihat hasil
data

--2025-07-17 17:21:34--  http://data/stopwords_id.txt
Resolving data (data)... failed: No address associated with hostname.
wget: unable to resolve host address ‘data’
--2025-07-17 17:21:34--  https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/stopwords%20list.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7709 (7.5K) [text/plain]
Saving to: ‘raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/stopwords list.txt’

raw.githubuserconte 100%[===================>]   7.53K  --.-KB/s    in 0s      

2025-07-17 17:21:34 (77.1 MB/s) - ‘raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/stopwords list.txt’ saved [7709/7709]

FINISHED --2025-07-17 17:21:34--
Total wall clock time

/tmp/ipython-input-21-2415590393.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['no_stopwords'] = data['cleaned_text2'].apply(remove_custom_stopwords)


,published_at,keyword,video_title,comment_text,label,cleaned_text,normalized_comment,cleaned_text2,no_stopwords
0,2025-07-09 07:39:42,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Starbucks dukung LGBT.mari boikot,3,starbucks dukung lgbt mari boikot,starbucks dukung lgbt mari boikot,starbucks dukung lgbt mari boikot,starbucks dukung lgbt mari boikot
1,2025-06-21 08:28:00,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"starbug itu kopi shop apa an sich,gk jelas.......",3,starbug itu kopi shop apa an sich gk jelas pin...,starbucks itu kopi shop apa an sih tidak jelas...,starbucks itu kopi shop apa an sih tidak jelas...,starbucks kopi shop apa an tidak jelas pengen ...
2,2025-06-20 00:33:48,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,"imigran timteng memang gila, berhasil menarik ...",2,imigran timteng memang gila berhasil menarik s...,imigran timurtengah memang gila berhasil menar...,imigran timurtengah memang gila berhasil menar...,imigran timurtengah memang gila berhasil menar...
3,2025-06-10 17:17:52,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Kmu beli satu kopi sama aja mengirim 1 peluru ...,3,kmu beli satu kopi sama aja mengirim peluru ke...,kamu beli satu kopi sama saja mengirim peluru ...,kamu beli satu kopi sama saja mengirim peluru ...,kamu beli satu kopi sama saja mengirim peluru ...
4,2025-03-21 01:15:54,#boikotstarbuck,Starbucks Oleng. Karena Boikot? Schultz Bilang...,Saya nggak tau bagaimana kok Starbucks bisa gi...,3,saya nggak tau bagaimana kok starbucks bisa gi...,saya tidak tahu bagaimana sih starbucks bisa b...,saya tidak tahu bagaimana sih starbucks bisa b...,saya tidak tahu bagaimana starbucks bisa begit...
...,...,...,...,...,...,...,...,...,...
6147,2024-04-30 17:29:06,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Pertanyaanya Indonesia kapan,2,pertanyaanya indonesia kapan,pertanyaanya indonesia kapan,pertanyaanya indonesia kapan,pertanyaanya kapan
6148,2024-04-30 16:25:40,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Indo apa kabar ? Masih suka kah melihat anak2 ...,2,indo apa kabar masih suka kah melihat anak tek...,indonesia apa kabar masih suka kah melihat ana...,indonesia apa kabar masih suka kah melihat ana...,apa kabar masih suka melihat anak bersalah dib...
6149,2024-04-30 16:04:55,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,❤🇮🇱🇮🇱❤ ane boikot makanan dan restoran palestina,3,ane boikot makanan dan restoran palestina,saya boikot makanan dan restoran palestina,saya boikot makanan dan restoran palestina,saya boikot makanan restoran palestina
6150,2024-04-30 16:01:26,#boikotkfc,KFC Malaysia Tutup 100 Gerai Imbas Aksi Boikot...,Orang +62 nggak malu sama Malaysia yang udah b...,3,orang nggak malu sama malaysia yang udah beran...,orang tidak malu sama malaysia yang sudah bera...,orang tidak malu sama malaysia yang sudah bera...,orang tidak malu sama malaysia yang sudah bera...


###Data Lemmatization

In [22]:
-

SyntaxError: invalid syntax (ipython-input-22-476313318.py, line 1)

In [ ]:
# Proses mengubah kata ke bentuk dasarnya (root word).
# Inisialisasi stemmer Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming menggunakan Sastrawi
def stem_text(text):
    return stemmer.stem(text)

# Terapkan stemming pada kolom 'no_stopwords'
data['stemmed_text'] = data['no_stopwords'].apply(stem_text)

data

### Stopword lanjutan

In [ ]:
# --- Kode untuk Top 100 Kata dari df_stemmed ---

from collections import Counter

all_words = []

# Iterasi melalui setiap entri di kolom 'stemmed_text'
# Menggunakan .dropna() untuk menghindari error jika ada nilai NaN
# Menggunakan .astype(str) untuk mengubah semua nilai menjadi string (mengantisipasi jika ada tipe data lain)
for text_entry in data['stemmed_text'].dropna().astype(str):
    if text_entry.strip():  # Memastikan string tidak kosong setelah di-strip
        # Jika isi 'stemmed_text' adalah string kata-kata yang dipisahkan spasi
        words = text_entry.split()
        all_words.extend(words)
        # Jika isi 'stemmed_text' sudah berupa list of words, Anda bisa langsung:
        # all_words.extend(text_entry) # Namun, ini memerlukan text_entry sudah pasti list

# Hitung frekuensi setiap kata
word_counts = Counter(all_words)

# Dapatkan 100 kata paling umum
top_100_words = word_counts.most_common(100)

print("--- Top 100 kata yang sering muncul dari 'stemmed_text' ---")
if top_100_words:
    for word, count in top_100_words:
        print(f"{word}: {count}")
else:
    print("Tidak ada kata untuk dihitung (mungkin semua entri kosong atau NaN).")
print("-" * 40)

In [ ]:
# Tambahkan stopwords tambahan
additional_stopwords = {
    'adik', 'agama', 'ajar', 'ala', 'amp', 'anak', 'anggota', 'anggar', 'antem', 'apa', 'arti', 'asasi', 'bantu',
    'bawa', 'beda', 'bicara', 'bikin', 'bilang', 'bina', 'bukti', 'cari', 'coba', 'contoh', 'daerah', 'dewan',
    'diam', 'didik', 'disiplin', 'dukung', 'gaji buta', 'generasi', 'generasi muda', 'gubenur', 'hak', 'hasil',
    'henti', 'hormat', 'ide', 'ikut', 'jakarta', 'jalan', 'jaman', 'jabatan', 'jabat', 'kak seto', 'kali',
    'karakter', 'keluarga', 'kerja', 'kaya', 'kirim', 'khofifah', 'komentar', 'komisi', 'koruptor', 'lindung',
    'lihat', 'lembaga', 'lulus', 'langsung', 'malu', 'makan', 'makan gaji', 'manusia', 'masuk', 'masyarakat',
    'media', 'metropolitan', 'metropolitan televisi', 'mental', 'moga', 'mu', 'muda', 'murid', 'nama', 'nasional',
    'neng', 'negara', 'nilai', 'nol', 'nyata', 'omong', 'omon', 'omon omon', 'ono', 'otak', 'pas', 'penuh',
    'pikir', 'pilih', 'pimpin', 'pintar', 'republik', 'remaja', 'rg', 'rakyat', 'rumah', 'sila', 'sehat', 'sekolah',
    'sesuaI', 'si', 'smp', 'solusi', 'swt', 'tanggung', 'televisi', 'teman', 'tentara', 'teori', 'tinggal', 'timur',
    'tuju', 'tujuan', 'tuju', 'ubah', 'uang', 'umpat', 'umpat umpat', 'urus', 'wakil', 'wajib', 'warga', 'wudele',
    'nama', 'rocky'
}

# Gabungkan dengan stopwords yang sudah ada
id_stop = set(id_stop).union(additional_stopwords)

# Fungsi hapus stopwords dari list kata
def remove_custom_stopwords(text):
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in id_stop])

# Terapkan ke kolom hasil lemmatization (yang sudah berupa list kata)
data['no_stopwords2'] = data['stemmed_text'].apply(remove_custom_stopwords)

data

In [ ]:
# --- Kode untuk Top 100 Kata dari df_stemmed ---

from collections import Counter

all_words = []

# Iterasi melalui setiap entri di kolom 'stemmed_text'
# Menggunakan .dropna() untuk menghindari error jika ada nilai NaN
# Menggunakan .astype(str) untuk mengubah semua nilai menjadi string (mengantisipasi jika ada tipe data lain)
for text_entry in data['no_stopwords2'].dropna().astype(str):
    if text_entry.strip():  # Memastikan string tidak kosong setelah di-strip
        # Jika isi 'stemmed_text' adalah string kata-kata yang dipisahkan spasi
        words = text_entry.split()
        all_words.extend(words)
        # Jika isi 'stemmed_text' sudah berupa list of words, Anda bisa langsung:
        # all_words.extend(text_entry) # Namun, ini memerlukan text_entry sudah pasti list

# Hitung frekuensi setiap kata
word_counts = Counter(all_words)

# Dapatkan 100 kata paling umum
top_100_words = word_counts.most_common(100)

print("--- Top 100 kata yang sering muncul dari 'stemmed_text' ---")
if top_100_words:
    for word, count in top_100_words:
        print(f"{word}: {count}")
else:
    print("Tidak ada kata untuk dihitung (mungkin semua entri kosong atau NaN).")
print("-" * 40)

### Tokenisasi

In [ ]:
# Proses memecah teks menjadi bagian-bagian kecil yang disebut token. Token biasanya berupa kata, frasa, atau simbol.
# Fungsi untuk tokenisasi teks
def tokenize_text(text):
    return word_tokenize(text)  # Tokenisasi teks menjadi list kata

# Terapkan tokenisasi ke kolom 'stemmed_text'
data['tokenize_text'] = data['no_stopwords2'].apply(tokenize_text)

data

## Ekspor Data Clean

In [ ]:
# Simpan ke CSV (opsional)
#data.to_csv('final_preprocessed_text_Full.csv', index=False)

Data diekspor dan akan diunggah ke GitHub untuk memudahkan proses visualisasi, sehingga tidak perlu menjalankan ulang seluruh proses dari data mentah.

# Visualisasi dan Interpretasi


**Import Data Clean**

Berikut link github setelah di lakukan cleaning dari data mentah:

https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/final_preprocessed_text_Full.csv

**Import data yang telah di ekspor**
```
import pandas as pd
file_ = 'data/data_final_all_fix.csv'
try: #Loading Locally
    df_final = pd.read_csv(file_)
except Exception as err_:
    print(err_," Trying to load data from GitHub.")
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/final_preprocessed_text_Full.csv

print(df_final.shape)
df_final.sample(10)

```


In [ ]:
import pandas as pd

file_ = 'data/data_final_all_fix.csv'
try: #Loading Locally
    df_final = pd.read_csv(file_)
except Exception as err_:
    print(err_, "Trying to load data from GitHub.")
    !mkdir -p data
    !wget -P data/ https://raw.githubusercontent.com/awaliaaaa23/Final-Project-AMS-2025/refs/heads/main/final_preprocessed_text_Full.csv

    # Tambahkan ini: baca file hasil unduhan
    df_final = pd.read_csv("data/final_preprocessed_text_Full.csv")

print(df_final.shape)
df_final.sample(10)


In [ ]:
df_final = data

In [ ]:
# Jumlah Komentar
print("Jumlah komen yang kami peroleh:", df_final.shape[0],"komen")

In [ ]:
df_final.info()

In [ ]:
# Menghapus kolom yang tidak digunakan
# df_final = df_final.drop(columns=['cleaned_text', 'normalized_comment', 'cleaned_text2', 'no_stopwords', 'stemmed_text','no_stopwords2'])


# Mapping angka ke label sentimen
sentiment_mapping = {1: 'Positif', 2: 'Netral', 3: 'Negatif'}

# Menambahkan kolom label sentimen berdasarkan mapping
df_final['sentiment_label'] = df_final['label'].map(sentiment_mapping)

# Mengonversi kolom 'published_at' ke format datetime
df_final['published_at'] = pd.to_datetime(df_final['published_at'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Mengurutkan data berdasarkan Waktu
df_final = df_final.sort_values(by='published_at')
df_final.reset_index(drop=True, inplace=True)

## Distribusi Sentimen Berdasarkan Keyword


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Crosstab keyword vs label
cross_tab = pd.crosstab(df_final['keyword'], df_final['label'])

# Pakai tab10 yang terang dan aman di mata
ax = cross_tab.plot(
    kind='bar',
    stacked=True,
    figsize=(10, 6),
    color=plt.get_cmap('tab10').colors
)

plt.xticks(rotation=45)
plt.xlabel('Keyword')
plt.ylabel('Jumlah Komentar')
plt.title('Distribusi Sentimen Berdasarkan Keyword', fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
cross_tab

Berdasarkan grafik, keyword #aksiboikot, #boikotstarbuck, #boikotunilever, #dampakboikot dan #penggantiprodukisraeldidominasi oleh komentar dengan sentimen negatif terhadap brand, yang berarti publik menunjukkan dukungan terhadap aksi boikot. Sementara itu, #boikotkfc, #boikotprodukisrael, dan #produkisrael didominasi oleh sentimen netral. Hal ini menunjukkan bahwa percakapan pada ketiga keyword tersebut cenderung bersifat informatif, mempertanyakan, atau belum secara tegas menunjukkan sikap mendukung maupun menolak boikot.

## Keyword Terpopuler Berdasarkan Jumlah Komentar

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Hitung total komentar per keyword
total_per_keyword = cross_tab.sum(axis=1).sort_values(ascending=False)

plt.figure(figsize=(10, 6))

# Membuat gradasi warna viridis sesuai jumlah keyword
colors = sns.color_palette("viridis", len(total_per_keyword))

total_per_keyword.plot(
    kind='barh',
    color=colors,
    edgecolor='black'
)

plt.xlabel('Total Komentar')
plt.ylabel('Keyword')
plt.title('Keyword Terpopuler Berdasarkan Jumlah Komentar')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


Grafik ini menunjukkan bahwa keyword #boikotprodukisrael, #boikotkfc, #produkisrael memiliki jumlah keyword terbanyak. Hal ini mengindikasikan bahwa ketiga keyword tersebut menjadi fokus utama dalam percakapan publik terkait aksi boikot, baik sebagai bentuk dukungan maupun diskusi seputar isu tersebut.Sementara itu, keyword seperti #dampakboikot, #penggantiprodukisrael, dan #boikotstarbuck berada di tingkat menengah, yang menunjukkan bahwa isu tersebut tetap relevan, namun tidak sebesar topik utama.
Adapun #boikotunilever dan #aksiboikot memiliki jumlah komentar paling sedikit, yang bisa terjadi karena hanya ramai dalam waktu yang singkat.

## Tren Komentar Bulanan

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pastikan tipe datetime tanpa cek manual
df_final['published_at'] = pd.to_datetime(df_final['published_at'], errors='coerce').fillna(pd.Timestamp('2000-01-01'))

df_final['bulan'] = df_final['published_at'].dt.to_period('M').astype(str)

top_keywords = df_final['keyword'].value_counts().head(10).index.tolist()

palette = sns.color_palette("tab10", n_colors=10)

for idx, keyword in enumerate(top_keywords):
    keyword_data = df_final[df_final['keyword'] == keyword].groupby('bulan').size()

    plt.figure(figsize=(8, 4))
    plt.plot(keyword_data.index, keyword_data.values, marker='o', linewidth=2,
             color=palette[idx])

    plt.title(f"Tren Komentar Bulanan: {keyword}", fontsize=14, fontweight='bold')
    plt.xlabel('Bulan', fontsize=12)
    plt.ylabel('Jumlah Komentar', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()


Percakapan mengenai boikot mulai ramai sejak November–Desember 2023, seiring dengan munculnya isu serangan Israel ke Gaza. Beberapa keyword seperti #boikotprodukisrael, #produkisrael, dan #boikotkfc terus diperbincangkan hingga pertengahan 2025, menandakan bahwa isu ini bukan sekadar tren sesaat, melainkan tetap menarik perhatian publik dalam jangka waktu panjang. Sementara itu, keyword seperti #penggantiprodukisrael, #dampakboikot, #boikotunilever, dan #boikotstarbuck dan #aksiboikot hanya ramai di awal, kemudian menurun dan tidak lagi banyak dibahas. Pola ini menunjukkan bahwa sebagian besar isu cenderung bersifat musiman atau sekadar mengikuti momentum, namun ada juga isu yang tetap relevan dan konsisten muncul di percakapan publik.

## Jumlah Komentar negatif per keyword

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter data negatif
negatif = df_final[df_final['label'] == 3]
top_keywords = negatif['keyword'].value_counts().head(10)

plt.figure(figsize=(10, 6))

# Warna tegas yang konsisten, bukan gradasi lembut
colors = sns.color_palette("plasma", n_colors=len(top_keywords))

top_keywords.plot(
    kind='bar',
    color=colors,
    edgecolor='black'
)

plt.ylabel('Jumlah Komentar Negatif')
plt.title('Jumlah Komentar Negatif per Keyword', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()


Berdasarkan grafik, keyword #produkisrael memiliki jumlah komentar negatif tertinggi dibandingkan keyword lain, diikuti oleh #dampakboikot dan #boikotkfc. Meskipun pada analisis sebelumnya #produkisrael dan #boikotkfc didominasi oleh sentimen netral, jumlah komentar negatifnya tetap yang paling besar. Hal ini menunjukkan #produkisrael dan #boikotkfc menjadi topik yang paling banyak menarik perhatian publik, baik dari yang bersikap netral maupun yang aktif menunjukkan sikap negatif (mendukung boikot).

##Durasi waktu percakapan setiap keyword

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_final['published_at'] = pd.to_datetime(df_final['published_at'], errors='coerce')
top_10_keywords = df_final['keyword'].value_counts().head(10).index.tolist()

durasi = df_final.groupby('keyword')['published_at'].agg(['min', 'max'])
durasi['lama_hari'] = (durasi['max'] - durasi['min']).dt.days

top_durasi = durasi.loc[top_10_keywords].sort_values('lama_hari', ascending=False)

# Pakai gradasi magma
colors = sns.color_palette("magma", n_colors=len(top_durasi))

plt.figure(figsize=(8, 5))
plt.barh(
    top_durasi.index,
    top_durasi['lama_hari'],
    color=colors,
    edgecolor='black'
)

plt.xlabel('Durasi Percakapan (Hari)', fontsize=12)
plt.ylabel('Keyword', fontsize=12)
plt.title('Durasi Waktu Percakapan per Keyword', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()


Grafik menunjukkan bahwa percakapan terkait keyword #produkisrael, #boikotprodukisrael, dan #penggantiprodukisrael memiliki durasi percakapan paling lama. Ini mengindikasikan bahwa isu terkait produk Israel konsisten menjadi perhatian publik dalam jangka waktu yang lama. Sementara itu, keyword seperti #boikotkfc dan #boikotstarbuck memiliki durasi percakapan yang lebih pendek, menunjukkan isu tersebut cenderung bersifat musiman atau hanya ramai saat momen-momen tertentu.

## Tren Percakapan Bulanan

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Pastikan kolom datetime dan bulan
df_final['published_at'] = pd.to_datetime(df_final['published_at'])
df_final['bulan'] = df_final['published_at'].dt.to_period('M').astype(str)

# Ambil 10 keyword terbanyak
top_keywords = df_final['keyword'].value_counts().head(10).index.tolist()

# Warna kontras dan konsisten
sentiment_colors = {
    1: '#2ca02c',  # Hijau (Positif)
    2: '#ff7f0e',  # Oranye (Netral)
    3: '#d62728'   # Merah (Negatif)
}

# Label sentimen mapping
label_map = {1: 'Positif', 2: 'Netral', 3: 'Negatif'}

# Visualisasi per keyword
for keyword in top_keywords:
    keyword_df = df_final[df_final['keyword'] == keyword]
    monthly_sentiment = keyword_df.groupby(['bulan', 'label']).size().unstack(fill_value=0)

    plt.figure(figsize=(10, 5))
    for label in sorted(monthly_sentiment.columns):
        plt.plot(
            monthly_sentiment.index,
            monthly_sentiment[label],
            marker='o',
            linewidth=2,
            label=label_map[label],
            color=sentiment_colors[label]
        )

    plt.title(f"Tren Percakapan Bulanan Berdasarkan Sentimen: {keyword}", fontsize=14, fontweight='bold')
    plt.xlabel("Bulan", fontsize=12)
    plt.ylabel("Jumlah Komentar", fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.legend(title="Sentimen", fontsize=10)
    plt.tight_layout()
    plt.show()


Beberapa keyword mengalami lonjakan pada November dan Desember 2023, umumnya didominasi oleh sentimen negatif (mendukung boikot), sebagai respons terhadap serangan Israel ke Gaza yang memicu aksi boikot terhadap berbagai merek dan produk. Tagar seperti #boikotunilever dan #boikotstarbuck menunjukkan lonjakan sentimen negatif yang berulang, menandakan bahwa isu boikot terhadap merek tersebut memicu emosi publik di waktu tertentu. Di sisi lain, beberapa tagar seperti #produkisrael, #boikotkfc, dan #boikotprodukisrael menunjukkan lonjakan sentimen netral yang cukup tinggi pada periode tertentu. Ini mengindikasikan bahwa sebagian percakapan publik bersifat informatif atau tidak berpihak secara emosional.

Sementara itu, pada tagar #dampakboikot, sentimen positif justru mencapai puncaknya pada November 2023, yang mengindikasikan adanya opini yang menilai boikot tidak berdampak signifikan atau tetap memilih menggunakan produk yang diboikot.


# Analisis Sentimen

## Pie chart sentimen

In [ ]:
import matplotlib.pyplot as plt

# Menghitung distribusi kategori
category_counts = data['sentiment_label'].value_counts()

# Membuat pie chart
category_counts.plot.pie(autopct='%1.1f%%', colors=['lightgreen', 'skyblue', 'lightcoral'])

# Menambahkan judul
plt.title('Distribusi Kategori')

# Menampilkan chart
plt.ylabel('')  # Menghilangkan label y agar lebih rapi
plt.show()

Interpretasi
Netral (43.5%) :
- Mayoritas komentar berisi opini dan pertanyaan apakah suatu produk juga harus diboikot. Selain itu, kritik untuk para penyeru aksi boikot juga banyak ditemukan.
- Netralitas muncul karena kebingungan netizen terhadap kebenaran informasi terkait aksi boikot produk.  

Negatif (32.7%) :
-

## Sentimen Positif

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import networkx as nx
import matplotlib.pyplot as plt

# Pastikan df_stemmed memiliki kolom 'text'
# Ambil daftar teks (kalimat) dan ubah ke lowercase
texts = data[data['label'] == 1]['no_stopwords2'].astype(str).str.lower().tolist()

# Gunakan CountVectorizer untuk ekstraksi bigram
vectorizer = CountVectorizer(ngram_range=(2, 2), min_df=2)  # hanya bigram yang muncul minimal 2x
X = vectorizer.fit_transform(texts)
bigrams = vectorizer.get_feature_names_out()

# Pisahkan setiap bigram menjadi pasangan kata
edges = [tuple(b.split()) for b in bigrams]

# Buat Graph dari bigram
G = nx.Graph()
G.add_edges_from(edges)

# Visualisasi graf bigram
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, k=0.5, seed=42)  # layout posisi node
nx.draw(
    G, pos,
    with_labels=True,
    node_color='lightblue',
    edge_color='gray',
    node_size=1000,
    font_size=10,
    font_weight='bold'
)
plt.title("Word Link (Bigram Network) dari Teks yang Sudah Distem", fontsize=16)
plt.show()

In [ ]:
# Cek kata apa yang paling banyak punya edge (koneksi).
import networkx as nx

centrality = nx.degree_centrality(G)
sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:10]

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Filter data ke label positif
positif_data = data[data['label'] == 1]

# Gabungkan semua token menjadi satu list
all_tokens = positif_data['tokenize_text'].sum()  # karena isinya list per baris

# Ubah list token menjadi string dipisahkan spasi
text_positif = " ".join(all_tokens)

# Buat Word Cloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    max_words=200,
    colormap='YlGnBu'
).generate(text_positif)

# Tampilkan
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud dari Token untuk Komentar Positif", fontsize=16)
plt.show()

Kata “boikot” dan “israel” tetap muncul besar, tapi konteksnya bukan mendukung aksi boikot melainkan menyindir orang yang boikot.

Kata “munafik”, “phk”, “usaha”, “pekerja”, “ekonomi”, “hidup”, “karyawan”
menunjukkan kekhawatiran atau sindiran bahwa boikot justru merugikan ekonomi ekonomi dan tenaga kerja Indonesia yang berpotensi adanya pemutusan hubungan kerja (PHK).

Kata “starbucks”, “kfc”, “unilever”, “youtube”, “google”
menunjukan sarkasme pada kontradiksi dimana netizen teriak boikot tapi tetap pakai produk buatan mereka.



## Sentimen Negatif

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import networkx as nx
import matplotlib.pyplot as plt

# Pastikan df_stemmed memiliki kolom 'text'
# Ambil daftar teks (kalimat) dan ubah ke lowercase
texts = data[data['label'] == 3]['no_stopwords2'].astype(str).str.lower().tolist()

# Gunakan CountVectorizer untuk ekstraksi bigram
vectorizer = CountVectorizer(ngram_range=(2, 2), min_df=2)  # hanya bigram yang muncul minimal 2x
X = vectorizer.fit_transform(texts)
bigrams = vectorizer.get_feature_names_out()

# Pisahkan setiap bigram menjadi pasangan kata
edges = [tuple(b.split()) for b in bigrams]

# Buat Graph dari bigram
G = nx.Graph()
G.add_edges_from(edges)

# Visualisasi graf bigram
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, k=0.5, seed=42)  # layout posisi node
nx.draw(
    G, pos,
    with_labels=True,
    node_color='lightblue',
    edge_color='gray',
    node_size=1000,
    font_size=10,
    font_weight='bold'
)
plt.title("Word Link (Bigram Network) dari Teks yang Sudah Distem", fontsize=16)
plt.show()

In [ ]:
# Cek kata apa yang paling banyak punya edge (koneksi).
import networkx as nx

centrality = nx.degree_centrality(G)
sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:10]

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Filter data ke label positif
positif_data = data[data['label'] == 3]

# Gabungkan semua token menjadi satu list
all_tokens = positif_data['tokenize_text'].sum()  # karena isinya list per baris

# Ubah list token menjadi string dipisahkan spasi
text_positif = " ".join(all_tokens)

# Buat Word Cloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    max_words=200,
    colormap='YlGnBu'
).generate(text_positif)

# Tampilkan
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud dari Token untuk Komentar Negatif", fontsize=16)
plt.show()

Dari word cloud komentar negatif, terlihat kata-kata dominan seperti boikot, Produk Israel, kfc, unilever dan Starbucks. Ini menunjukkan ketidakinginan netizen untuk membeli brand yang dianggap terafiliasi dengan israel.

Disisi lain muncul juga kata UMKM, produk lokal, ganti, saing, enak, harga dan murah menunjukkan bahwa alasan konsumen berpindah ke produk lokal bukan hanya soal ketidakterikatan brand terhadap isu palestina tetapi juga mengenai harga dan kualitas.

## Sentimen Netral

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import networkx as nx
import matplotlib.pyplot as plt

# Pastikan df_stemmed memiliki kolom 'text'
# Ambil daftar teks (kalimat) dan ubah ke lowercase
texts = data[data['label'] == 2]['no_stopwords2'].astype(str).str.lower().tolist()

# Gunakan CountVectorizer untuk ekstraksi bigram
vectorizer = CountVectorizer(ngram_range=(2, 2), min_df=2)  # hanya bigram yang muncul minimal 2x
X = vectorizer.fit_transform(texts)
bigrams = vectorizer.get_feature_names_out()

# Pisahkan setiap bigram menjadi pasangan kata
edges = [tuple(b.split()) for b in bigrams]

# Buat Graph dari bigram
G = nx.Graph()
G.add_edges_from(edges)

# Visualisasi graf bigram
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, k=0.5, seed=42)  # layout posisi node
nx.draw(
    G, pos,
    with_labels=True,
    node_color='lightblue',
    edge_color='gray',
    node_size=1000,
    font_size=10,
    font_weight='bold'
)
plt.title("Word Link (Bigram Network) dari Teks yang Sudah Distem", fontsize=16)
plt.show()

In [ ]:
# Cek kata apa yang paling banyak punya edge (koneksi).
import networkx as nx

centrality = nx.degree_centrality(G)
sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:10]

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Filter data ke label positif
positif_data = data[data['label'] == 2]

# Gabungkan semua token menjadi satu list
all_tokens = positif_data['tokenize_text'].sum()  # karena isinya list per baris

# Ubah list token menjadi string dipisahkan spasi
text_positif = " ".join(all_tokens)

# Buat Word Cloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    max_words=200,
    colormap='YlGnBu'
).generate(text_positif)

# Tampilkan
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud dari Token untuk Komentar Netral", fontsize=16)
plt.show()

Dari word cloud komentar netral, terlihat kemunculan kata-kata seperti “produk”, “harga”, “boikot”, “pengganti”, dan “lokal”. Ini menunjukkan bahwa netizen hanya berdiskusi terkait perbandingan harga produk yang termasuk list boikot dengan produk lokal tanpa menganjurkan untuk memboikot ataupun mendukung produk yang diboikot.

## Analisis Lebih Lanjut Sentimen Netral


In [ ]:
# Misal kolom sentimen bernama 'label'
netral_data = data[data['label'] == 2]
print(f"Total komentar netral: {len(netral_data)}")

In [ ]:
from collections import Counter

# Gabungkan semua teks komentar netral
all_text = ' '.join(netral_data['no_stopwords2'])

# Pecah jadi kata
words = all_text.split()
word_freq = Counter(words).most_common(20)

# Visualisasi
import matplotlib.pyplot as plt
words, freqs = zip(*word_freq)
plt.figure(figsize=(10,5))
plt.barh(words[::-1], freqs[::-1], color='slategray')
plt.title('20 Kata Paling Sering di Komentar Netral')
plt.xlabel('Frekuensi')
plt.tight_layout()
plt.show()

In [ ]:
# Hitung total komentar per video
total_per_video = data['video_title'].value_counts().reset_index(name='total_komentar')
total_per_video.columns = ['video_title', 'total_komentar']

# Hitung komentar netral per video
netral_per_video = netral_data['video_title'].value_counts().reset_index(name='komentar_netral')
netral_per_video.columns = ['video_title', 'komentar_netral']

# Gabungkan berdasarkan video_title
summary = pd.merge(netral_per_video, total_per_video, on='video_title')

# Hitung rasio komentar netral
summary['rasio_netral'] = summary['komentar_netral'] / summary['total_komentar']

# Tampilkan top 10 video dengan rasio netral tertinggi
summary.sort_values('rasio_netral', ascending=False).head(10)

# Topik Modelling

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic

## Sentimen Positif

In [ ]:
text_pos = data[data['label'] == 1]
texts = text_pos['no_stopwords2']
texts

In [ ]:
texts = texts.astype(str).tolist()

In [ ]:
# Install UMAP if not already installed
!pip install umap-learn
!pip install hdbscan # Install hdbscan library

# Import UMAP
from umap import UMAP
from hdbscan import HDBSCAN # Import HDBSCAN

In [ ]:
# Inisialisasi model UMAP dengan random_state
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=37)

# Inisialisasi model clustering HDBSCAN dengan random_state
hdbscan_model = HDBSCAN(min_cluster_size=10,
                        metric='euclidean',
                        cluster_selection_method='eom',
                        prediction_data=True)

# Inisialisasi BERTopic dengan model yang sudah disetel random_state
topic_model = BERTopic(
    language="indonesian",
    calculate_probabilities=True,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True
)

# Fit model
topics, probs = topic_model.fit_transform(texts)

# Tampilkan topik dominan
topic_model.get_topic_info()

In [ ]:
topic_model.visualize_topics()

## Sentimen Negatif

In [ ]:
text_pos2 = data[data['label'] == 3]
texts2 = text_pos2['no_stopwords2']
texts2

In [ ]:
texts2 = texts2.astype(str).tolist()

In [ ]:
topic_model = BERTopic(language="indonesian", calculate_probabilities=True)  # gunakan bahasa Indonesia
topics, probs = topic_model.fit_transform(texts2)

topic_model.get_topic_info()

In [ ]:
topic_model.visualize_barchart(top_n_topics=3)

## Sentimen Netral

In [ ]:
text_pos3 = data[data['label'] == 2]
texts3 = text_pos3['no_stopwords2']
texts3

In [ ]:
texts3 = texts3.astype(str).tolist()

In [ ]:
# Inisialisasi model UMAP dengan random_state
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=37)

# Inisialisasi model clustering HDBSCAN dengan random_state
hdbscan_model = HDBSCAN(min_cluster_size=10,
                        metric='euclidean',
                        cluster_selection_method='eom',
                        prediction_data=True)

# Inisialisasi BERTopic dengan model yang sudah disetel random_state
topic_model3 = BERTopic(
    language="indonesian",
    calculate_probabilities=True,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True
)

# Fit model
topics, probs = topic_model3.fit_transform(texts3)

# Tampilkan topik dominan
topic_model3.get_topic_info()

In [ ]:
topic_model3.visualize_topics()

# **Insight yang Didapat**

# **Rekomendasi**

# **Kesimpulan**

In [ ]:
sentiment_proportion = df_final['label'].value_counts()#(normalize=True)
sentiment_proportion